In [1]:
from natsort import natsorted
import glob
import pandas as pd

folder_path = '../docs/T6_m_chi/M_Phi'
file_paths = glob.glob(f'{folder_path}/*.csv')
file_paths = natsorted(file_paths)

dfs = []
for file_path in file_paths:
    df = pd.read_csv(file_path, names=['Phi', 'My'], sep=';')
    filename = file_path.split('/')[-1]  # Extract the filename from the file path
    df.columns.name = filename  # Set the column name as the filename
    df = df[df['Phi'] != 0]
    df['Phi'] = df['Phi']  # Convert Phi values to mrad
    df['My'] = df['My']*1000
    df['spring'] = df['My'] / df['Phi'] * 1000

    # Es braucht einen Nullpunkt
    df_null = pd.DataFrame([[0,0,0]], columns=df.columns)
    df = pd.concat([df, df_null], axis=0).sort_values("Phi").reset_index()
    
    dfs.append(df)


dfs[0]

M_Phi\M_Phi_qs0.csv,index,Phi,My,spring
0,0,-0.002840,-1.052005e+06,3.704027e+11
1,1,-0.002582,-1.048271e+06,4.059967e+11
2,2,-0.002560,-1.052005e+06,4.109394e+11
3,3,-0.001280,-1.030162e+06,8.048143e+11
4,4,-0.000640,-9.198105e+05,1.437204e+12
5,5,-0.000320,-6.713168e+05,2.097865e+12
6,6,-0.000160,-4.362859e+05,2.726787e+12
7,7,-0.000080,-3.137880e+05,3.922350e+12
8,8,-0.000040,-2.438395e+05,6.095988e+12
9,9,-0.000020,-1.907443e+05,9.537214e+12


In [2]:
import os

from RFEM.enums import NodalLoadDirection, CaseObjectType, MemberHingeNonlinearity
from RFEM.BasicObjects.member import Member
from RFEM.BasicObjects.node import Node
from RFEM.initModel import Model, Calculate_all
from RFEM.Loads.nodalLoad import NodalLoad
from RFEM.Results.resultTables import ResultTables
from RFEM.TypesForMembers.memberHinge import MemberHinge

# Modelinitiierung
Model(False, "T6_1D", delete=True)
Model.clientModel.service.begin_modification()



for j, iter in enumerate(dfs):

    diagram_around_y_table = Model.clientModel.factory.create('ns0:member_hinge.diagram_around_y_table')

    for i in range(len(iter)):
        mlvlp = Model.clientModel.factory.create('ns0:member_hinge_diagram_around_y_table_row')
        mlvlp.no = i+1
        mlvlp.row = Model.clientModel.factory.create('ns0:member_hinge_diagram_around_y_table')
        mlvlp.row.rotation = iter['Phi'].iloc[i]
        mlvlp.row.moment = iter['My'].iloc[i]
        # mlvlp.row.spring = iter['spring'].iloc[i]

        diagram_around_y_table.member_hinge_diagram_around_y_table.append(mlvlp)

        
    Spring_char = MemberHinge(no=j+1, params={
        "diagram_around_y_symmetric": "false",
        "moment_release_my_nonlinearity":"NONLINEARITY_TYPE_DIAGRAM",
        "diagram_around_y_start": "DIAGRAM_ENDING_TYPE_FAILURE",
        "diagram_around_y_end": "DIAGRAM_ENDING_TYPE_FAILURE",    
        "diagram_around_y_table":diagram_around_y_table,
        })
    
    Node(no=j+1, coordinate_X=j*0.1, coordinate_Y=0, coordinate_Z=0)
    Member(no=j+1, start_node_no=j+1, end_node_no=j+2, start_member_hinge_no=j+1)

    # print(f'Diagramm {dfs[j].columns.name}')

Model.clientModel.service.finish_modification()

Connecting to server...
Deleting results...


WebFault: b"Server raised fault: 'Objekt: Stab Nr. 137\nAttribut: Zugewiesen an Linie Nr.\nFehler: Die Definitionslinie ist nicht zul\xc3\xa4ssig.'"

In [ ]:
from suds.client import Client

client = Client('http://localhost:8081/wsdl')
new = client.service.get_active_model()+'wsdl'
model = Client(new)
print(model.service.get_member_hinge(1))


(member_hinge){
   no = 1
   user_defined_name_enabled = False
   name = "-D- DND | Lokales Koordinatensystem xyz | φ_y : Diagramm"
   members = "1"
   coordinate_system = "Local"
   axial_release_n = inf
   axial_release_vz = inf
   moment_release_my = 0.0
   axial_release_n_nonlinearity = "NONLINEARITY_TYPE_NONE"
   axial_release_vz_nonlinearity = "NONLINEARITY_TYPE_NONE"
   moment_release_my_nonlinearity = "NONLINEARITY_TYPE_DIAGRAM"
   partial_activity_along_x_negative_slippage = 0.0
   partial_activity_along_x_positive_slippage = 0.0
   partial_activity_along_y_negative_slippage = 0.0
   partial_activity_along_y_positive_slippage = 0.0
   partial_activity_along_z_negative_slippage = 0.0
   partial_activity_along_z_positive_slippage = 0.0
   partial_activity_around_x_negative_slippage = 0.0
   partial_activity_around_x_positive_slippage = 0.0
   partial_activity_around_z_negative_slippage = 0.0
   partial_activity_around_z_positive_slippage = 0.0
   diagram_around_y_symmetric = Fal